In [6]:
import sqlite3
import pandas as pd

In [7]:
db = sqlite3.connect('health_air.db')
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)

[('asthma',), ('air',)]


In [9]:
asthma = pd.read_sql_query("SELECT * from %s" % table_name, db)


,index,State,Year,Max AQI,Median AQI,Days Ozone,Days PM2.5,Days PM10
0,0,Alabama,2010,112,46,131,142,8
1,1,Alabama,2011,111,42,158,84,4
2,2,Alabama,2012,110,40,174,85,3
3,3,Alabama,2013,86,37,173,84,3
4,4,Alabama,2014,93,38,173,83,3
...,...,...,...,...,...,...,...,...
535,535,Wyoming,2015,110,34,235,29,42
536,536,Wyoming,2016,95,37,266,16,44
537,537,Wyoming,2017,169,38,259,18,42
538,538,Wyoming,2018,107,39,269,14,41
